In [58]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
import matplotlib.pyplot as plt
import seaborn as sns

In [59]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [60]:

# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels  # y_true
        self.prediction = self.predict()  # y_pred
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction

In [61]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1     # "impurity" - "нечистота", степень неопределенности
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [62]:
# Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выборки, ушедшей в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [63]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [64]:

# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        t_values = [row[index] for row in data]
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [65]:

# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels):

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [66]:
# Проход объекта по дереву для его классификации

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [95]:
# Предсказание деревом для всего датасета

def predict(data, tree):
    classes = []
    for obj in data.iterrows(): # for obj in data:
        prediction = classify_object(obj[1], tree)
        classes.append(prediction)
    return classes

In [75]:
# предсказание голосованием деревьев

def tree_vote(forest, data):

    # добавим предсказания всех деревьев в список
    predictions = []
    for tree in forest:
        predictions.append(predict(data, tree))
    
    # сформируем список с предсказаниями для каждого объекта
    predictions_per_object = list(zip(*predictions))
    
    # выберем в качестве итогового предсказания для каждого объекта то,
    # за которое проголосовало большинство деревьев
    voted_predictions = []
    for obj in predictions_per_object:
        voted_predictions.append(max(set(obj), key=obj.count))
        
    return voted_predictions

In [68]:
def custom_StandardScaler(x):
    res = (x - x.mean()) / x.std()
    return res

In [18]:
train_dataset = pd.read_csv('./data/train.csv', sep=',')
train_dataset.head()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
0,0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0
1,1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1
2,2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0
3,3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,1
4,4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0


In [19]:
train_dataset.dtypes

Id                       int64
age                    float64
years_of_experience    float64
lesson_price           float64
qualification          float64
physics                float64
chemistry              float64
biology                float64
english                float64
geography              float64
history                float64
mean_exam_points       float64
choose                   int64
dtype: object

In [20]:
train_dataset.describe()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,4999.50000,45.800900,1.974800,1702.440000,1.724300,0.37060,0.121500,0.117200,0.059100,0.02770,0.018000,64.435200,0.110900
std,2886.89568,8.030274,1.766883,523.789062,0.798845,0.48299,0.326724,0.321675,0.235824,0.16412,0.132958,13.595024,0.314024
min,0.00000,23.000000,0.000000,200.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,33.000000,0.000000
25%,2499.75000,40.000000,0.000000,1300.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,55.000000,0.000000
50%,4999.50000,46.000000,2.000000,1550.000000,2.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,63.000000,0.000000
75%,7499.25000,51.000000,3.000000,2150.000000,2.000000,1.00000,0.000000,0.000000,0.000000,0.00000,0.000000,74.000000,0.000000
max,9999.00000,68.000000,9.000000,3950.000000,4.000000,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000,100.000000,1.000000


In [21]:
len(train_dataset) - train_dataset.count()

Id                     0
age                    0
years_of_experience    0
lesson_price           0
qualification          0
physics                0
chemistry              0
biology                0
english                0
geography              0
history                0
mean_exam_points       0
choose                 0
dtype: int64

In [28]:
data = train_dataset.drop(['Id' , 'choose'], axis=1) 
data

,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0
1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0
2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0
3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0
4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0
5,37.0,3.0,1050.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0
6,54.0,3.0,800.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,52.0
7,32.0,2.0,2750.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,95.0
8,56.0,3.0,1300.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,58.0
9,44.0,4.0,2350.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,71.0


In [30]:
data = custom_StandardScaler(data)
data

,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
0,-1.345023,-1.117674,0.854466,0.345123,-0.767304,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,0.703552
1,0.771966,0.014262,-0.863783,0.345123,1.303134,-0.371874,2.744389,-0.250611,-0.168779,7.385804,-0.546906
2,-2.092195,0.580231,0.090800,-0.906684,1.303134,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,0.115101
3,-1.594080,0.580231,-1.245616,-0.906684,-0.767304,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,0.115101
4,0.024794,0.580231,1.045383,0.345123,1.303134,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,0.629995
5,-1.095965,0.580231,-1.245616,0.345123,-0.767304,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,-0.767575
6,1.021024,0.580231,-1.722907,-0.906684,-0.767304,2.688816,-0.364343,-0.250611,-0.168779,-0.135381,-0.914688
7,-1.718609,0.014262,1.999965,1.596931,1.303134,-0.371874,2.744389,-0.250611,-0.168779,-0.135381,2.248234
8,1.270081,0.580231,-0.768325,0.345123,1.303134,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,-0.473350
9,-0.224264,1.146199,1.236299,0.345123,-0.767304,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,0.482883


In [31]:
labels = train_dataset['choose']

In [32]:
task_train_data, task_test_data, task_train_labels, task_test_labels = tts(data, 
                                                                                     labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)


In [33]:
task_train_data = task_train_data.reset_index(drop=True)
task_train_labels = task_train_labels.reset_index(drop=True)

In [34]:
task_train_data_edit = task_train_data
task_train_labels_edit = task_train_labels 

In [38]:
task_train_data_edit = task_train_data_edit.rename(columns={x : y for x,y in zip(task_train_data_edit.columns, range(len(task_train_data_edit.columns)))})

In [98]:
task_test_data = task_test_data.reset_index(drop=True)
task_test_labels = task_test_labels.reset_index(drop=True)

In [99]:
task_test_data_edit = task_test_data
task_test_labels_edit = task_test_labels 

In [100]:
task_test_data_edit = task_test_data_edit.rename(columns={x : y for x,y in zip(task_test_data_edit.columns, range(len(task_train_test_edit.columns)))})

NameError: name 'task_train_test_edit' is not defined

In [54]:
task_train_data_edit

,0,1,2,3,4,5,6,7,8,9,10
0,1.270081,-0.551706,-0.386491,-0.906684,1.303134,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,0.556439
1,-0.722379,0.580231,1.713591,1.596931,-0.767304,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,1.733340
2,0.273851,-1.117674,-0.863783,-0.906684,1.303134,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,-0.473350
3,-0.722379,0.014262,-0.291033,-0.906684,-0.767304,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,-1.870920
4,2.017254,-0.551706,-0.959241,-0.906684,-0.767304,2.688816,-0.364343,-0.250611,-0.168779,-0.135381,-0.841131
5,2.141782,-1.117674,0.090800,0.345123,1.303134,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,0.262214
6,0.149322,0.580231,-0.100117,-0.906684,-0.767304,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,-0.473350
7,-0.099735,1.146199,2.668173,1.596931,1.303134,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,1.954009
8,0.149322,-0.551706,1.236299,1.596931,1.303134,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,1.586227
9,-0.348793,-0.551706,-0.672866,-0.906684,-0.767304,-0.371874,-0.364343,-0.250611,-0.168779,-0.135381,-0.546906


In [80]:
task_train_labels_edit

0       0
1       0
2       0
3       0
4       0
       ..
6995    0
6996    0
6997    0
6998    0
6999    0
Name: choose, Length: 7000, dtype: int64

In [69]:
%%time
import random
n_trees = 1
my_forest_1 = random_forest(task_train_data_edit, task_train_labels_edit,n_trees)

Wall time: 12min 30s


In [96]:
# Получим ответы для обучающей выборки 
task_train_answers_1 = tree_vote(my_forest_1, task_train_data_edit)

In [102]:
# Получим ответы для обучающей выборки 
task_test_answers_1 = tree_vote(my_forest_1, task_test_data_edit)

In [91]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [92]:
task_train_accuracy_1 = accuracy_metric(task_train_labels, task_train_answers_1)
print(f'Точность случайного леса из {n_trees} деревьев на обучающей выборке: {task_train_accuracy_1:.3f}')

Точность случайного леса из 1 деревьев на обучающей выборке: 90.186


In [106]:
task_train_accuracy_2 = accuracy_metric(task_test_labels, task_test_answers_1)
print(f'Точность случайного леса из {n_trees} деревьев на обучающей выборке: {task_train_accuracy_2:.3f}')

Точность случайного леса из 1 деревьев на обучающей выборке: 85.133
